# co-occurrence matrix
 - 共起行列

In [5]:
import numpy as np
import MeCab

In [6]:
def co_matrix(corpus, sep=' ', window_size = 1): #window_sizeを前後の文字をいくつ取るか
    word_to_id= {}
    id_to_word= {}
    cnt= 0
    #Wordにidを振る
    for co in corpus:
        for c in co.split(sep):
            if c not in word_to_id:
                word_to_id[c] = cnt
                id_to_word[cnt] = c
                cnt += 1   
                
    vocab_size = len(word_to_id)
    #出てきた単語数の正方行列を作成する。
    co_matrix = np.zeros((vocab_size , vocab_size)) #numpy.zeros(): 0で初期化（ゼロ埋め）
    for co in corpus:
        corpus_words = co.split(sep)
        #文章のword数をセット
        corpus_size = len(corpus_words)
        for idx, c in enumerate(corpus_words):
            #wordのidを取得する
            center_index = word_to_id[c]
            #window_sizeの数だけループ
            for diff in range(1 , window_size + 1):
                #対象の単語より前側を共起させる
                if idx - diff >= 0 :
                    left_word = corpus_words[idx - diff]
                    left_index = word_to_id[left_word]
                    co_matrix[center_index, left_index] += 1
                
                #対象の単語より後ろ側を共起させる
                if idx + diff < corpus_size:
                    right_word = corpus_words[idx + diff]
                    right_index = word_to_id[right_word]
                    co_matrix[center_index, right_index] += 1
    return word_to_id, id_to_word, co_matrix

In [7]:
co_matrix(['私 東京駅 行く 東京駅 山手線 乗る'])

({'私': 0, '東京駅': 1, '行く': 2, '山手線': 3, '乗る': 4},
 {0: '私', 1: '東京駅', 2: '行く', 3: '山手線', 4: '乗る'},
 array([[0., 1., 0., 0., 0.],
        [1., 0., 2., 1., 0.],
        [0., 2., 0., 0., 0.],
        [0., 1., 0., 0., 1.],
        [0., 0., 0., 1., 0.]]))

# 相互情報量

$$
\textrm{PMI} \left( x, y \right) = \log_{2} \frac{P \left(x,y \right)}{P \left(x \right) P \left(y \right)} \\
\textrm{PPMI} \left( x, y \right) = \max \left[0, \textrm{PMI}\left( x, y \right) \right]
$$

In [10]:
def ppmi(C, delta=1e-8):
    M = np.zeros_like(C, dtype=np.float32)
    N = np.sum(C)
    #単語ごとの個数の合計を求める。
    S = np.sum(C, axis=0)
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi = np.log(C[i, j] * N / S[i]*S[j] + delta)
            M[i , j] = max(0, pmi)
            
    return M

In [11]:
_, _,m = co_matrix(['私 東京駅 行く', '東京駅 山手線 乗る','東京駅 品川駅 東海道新幹線 一駅','東京駅 大宮 大宮 東北新幹線 乗り換える'], window_size=1)
print(m)
ppmi(m)

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 2. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


array([[0.       , 4.7004805, 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       ],
       [1.4816046, 0.       , 1.4816046, 2.1747518, 0.       , 2.1747518,
        0.       , 0.       , 2.867899 , 0.       , 0.       ],
       [0.       , 4.7004805, 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       ],
       [0.       , 4.0073333, 0.       , 0.       , 2.3978953, 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 3.7841897, 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       , 0.       ],
       [0.       , 4.0073333, 0.       , 0.       , 0.       , 0.       ,
        3.0910425, 0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 3.0910425,
        0.       , 2.3978953, 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.  